In [4]:
!pip3 install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 77.4 MB/s eta 0:00:00:00:0100:01


In [6]:
!git clone https://github.com/yanx27/Pointnet_Pointnet2_pytorch.git
%cd Pointnet_Pointnet2_pytorch

Cloning into 'Pointnet_Pointnet2_pytorch'...
remote: Enumerating objects: 842, done.
remote: Total 842 (delta 0), reused 0 (delta 0), pack-reused 842 (from 1)
Receiving objects: 100% (842/842), 68.77 MiB | 35.95 MiB/s, done.
Resolving deltas: 100% (485/485), done.
/kaggle/working/Pointnet_Pointnet2_pytorch


### Combine

In [23]:
import open3d as o3d
import numpy as np
import os

def combine_static_dynamic(static_ply_path, dynamic_ply_path, output_path):
    if not os.path.exists(static_ply_path):
        print(f"Ошибка: Файл {static_ply_path} не существует")
        return None
    if not os.path.exists(dynamic_ply_path):
        print(f"Ошибка: Файл {dynamic_ply_path} не существует")
        return None

    static_pcd = o3d.io.read_point_cloud(static_ply_path)
    static_points = np.asarray(static_pcd.points)
    static_colors = np.asarray(static_pcd.colors) if static_pcd.has_colors() else np.ones((len(static_points), 3)) * 0.7

    dynamic_pcd = o3d.io.read_point_cloud(dynamic_ply_path)
    dynamic_points = np.asarray(dynamic_pcd.points)
    dynamic_colors = np.asarray(dynamic_pcd.colors) if dynamic_pcd.has_colors() else np.ones((len(dynamic_points), 3)) * 0.3

    combined_points = np.vstack([static_points, dynamic_points])
    combined_colors = np.vstack([static_colors, dynamic_colors])

    combined_pcd = o3d.geometry.PointCloud()
    combined_pcd.points = o3d.utility.Vector3dVector(combined_points)
    combined_pcd.colors = o3d.utility.Vector3dVector(combined_colors)

    o3d.io.write_point_cloud(output_path, combined_pcd)
    print(f"Объединенный файл сохранен: {output_path}")

    return combined_pcd

In [25]:
combined_pcd = combine_static_dynamic(
    static_ply_path="/content/static_0000003106_0000003313-2.ply",
    dynamic_ply_path= "/content/dynamic_0000003106_0000003313.ply",
    output_path="combined_scene_3.ply"
)

Объединенный файл сохранен: combined_scene_3.ply


In [ ]:
# filtered = filter_dynamic_points_preserve_quality("/content/Pointnet_Pointnet2_pytorch/combined_scene.ply", "/content/best_model.pth", batch_size=16384*2)

### Visual by colors

In [ ]:
import torch
import numpy as np
import open3d as o3d
from models.pointnet2_sem_seg import get_model
import torch.nn.functional as F
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from collections import defaultdict

def visualize_dynamic_points_with_threshold(ply_file_path, model_path, output_file_path=None,
                                          threshold=0.5, device=('cuda' if torch.cuda.is_available() else "cpu"), voxel_size=0.1,
                                          use_downsample=True, edge_distance_threshold=5.0,
                                          z_upper_static_threshold=5.0, ground_height_threshold=1.0, grid_divisions=15):
    model = get_model(num_classes=2).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    pcd = o3d.io.read_point_cloud(ply_file_path)

    if use_downsample:
        pcd = pcd.voxel_down_sample(voxel_size)
        print(f"Downsample включён, voxel_size={voxel_size}")
    else:
        print("Downsample отключён")

    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.2, max_nn=30))

    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors) if pcd.has_colors() else np.ones((len(points), 3)) * 0.5
    normals = np.asarray(pcd.normals)

    print(f"Загружено точек после препроцессинга: {len(points)}")
    print(f"Используемый порог: {threshold}")

    batch_size = 200000
    all_dynamic_probs = []

    total_batches = (len(points) + batch_size - 1) // batch_size

    for batch_idx in range(total_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(points))

        batch_points = points[start_idx:end_idx]
        batch_colors = colors[start_idx:end_idx]
        batch_normals = normals[start_idx:end_idx]

        xyz = batch_points.copy()
        xyz_mean = np.mean(xyz, axis=0, keepdims=True)
        xyz_centered = xyz - xyz_mean

        xyz_max = np.max(np.abs(xyz_centered))
        if xyz_max > 0:
            xyz_normalized = xyz_centered / (xyz_max + 1e-8)
        else:
            xyz_normalized = xyz_centered

        rgb = batch_colors.copy()
        if np.max(rgb) > 1.0:
            rgb = rgb / 255.0

        rgb_normalized = (rgb - np.mean(rgb, axis=0, keepdims=True)) / (np.std(rgb, axis=0, keepdims=True) + 1e-8)

        features = np.hstack([xyz_normalized, rgb_normalized, batch_normals])

        features_tensor = torch.tensor(features, dtype=torch.float32).unsqueeze(0)
        features_tensor = features_tensor.permute(0, 2, 1).to(device)

        with torch.no_grad():
            pred, _ = model(features_tensor)
            probas = F.softmax(pred, dim=2)
            dynamic_probs = probas[0, :, 1].cpu().numpy()
            all_dynamic_probs.append(dynamic_probs)

        print(f"Обработан батч {batch_idx + 1}/{total_batches}")

    all_dynamic_probs = np.concatenate(all_dynamic_probs)

    pred_labels = (all_dynamic_probs > threshold).astype(int)

    min_x, min_y = np.min(points[:, 0]), np.min(points[:, 1])
    max_x, max_y = np.max(points[:, 0]), np.max(points[:, 1])

    x_edges = np.linspace(min_x, max_x, grid_divisions + 1)
    y_edges = np.linspace(min_y, max_y, grid_divisions + 1)

    ground_mask = np.zeros(len(points), dtype=bool)

    for i in range(grid_divisions):
        for j in range(grid_divisions):
            x_min, x_max = x_edges[i], x_edges[i+1]
            y_min, y_max = y_edges[j], y_edges[j+1]

            cell_mask = (points[:, 0] >= x_min) & (points[:, 0] < x_max) & \
                       (points[:, 1] >= y_min) & (points[:, 1] < y_max)

            if np.sum(cell_mask) == 0:
                continue

            dynamic_in_cell = cell_mask & (pred_labels == 1)

            if np.sum(dynamic_in_cell) > 0:
                min_z_cell = np.min(points[dynamic_in_cell, 2])
            else:
                min_z_cell = np.min(points[cell_mask, 2])

            ground_in_cell = cell_mask & (points[:, 2] < (min_z_cell + ground_height_threshold))
            ground_mask[ground_in_cell] = True

    # статика для пола
    pred_labels[ground_mask] = 0

    num_samples = min(1000, len(points))
    if num_samples > 0:
        np.random.seed(0)
        sample_idx = np.random.choice(len(points), num_samples, replace=False)
        sample_points_xy = points[sample_idx, :2]
        tree_sample = KDTree(sample_points_xy)
        dists, _ = tree_sample.query(sample_points_xy, k=2)
        avg_nn_dist = np.mean(dists[:, 1]) if num_samples > 1 else 0.0
    else:
        avg_nn_dist = 0.0



    grid_size = max(avg_nn_dist * 2.0, 1e-6)

    cell_points = defaultdict(list)
    for i in range(len(points)):
        p = points[i, :2]
        ix = int(p[0] / grid_size)
        iy = int(p[1] / grid_size)
        cell_points[(ix, iy)].append(i)

    occupied_cells = set(cell_points.keys())

    boundary_cells = set()
    for cell in occupied_cells:
        ix, iy = cell
        is_bound = False
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if dx == 0 and dy == 0:
                    continue
                neigh = (ix + dx, iy + dy)
                if neigh not in occupied_cells:
                    is_bound = True
                    break
            if is_bound:
                break
        if is_bound:
            boundary_cells.add(cell)

    is_boundary = np.zeros(len(points), dtype=bool)
    for cell in boundary_cells:
        for i in cell_points[cell]:
            is_boundary[i] = True

    # статика для пола для спорных
    non_ground_dynamic_idx = np.where((pred_labels == 1) & (~ground_mask))[0]
    if len(non_ground_dynamic_idx) > 0 and np.sum(is_boundary) > 0:
        boundary_idx = np.where(is_boundary)[0]
        boundary_tree = KDTree(points[boundary_idx, :2])
        dists_to_boundary, _ = boundary_tree.query(points[non_ground_dynamic_idx, :2])
        edge_mask = dists_to_boundary < edge_distance_threshold
        pred_labels[non_ground_dynamic_idx[edge_mask]] = 2
        print(f"Спорных точек определено: {np.sum(edge_mask)}")

    if z_upper_static_threshold is not None:
        min_z = np.min(points[:, 2])
        upper_mask = points[:, 2] > (min_z + z_upper_static_threshold)
        pred_labels[upper_mask] = 0

    print(f"Статических точек: {np.sum(pred_labels == 0)}")
    print(f"Динамических точек: {np.sum(pred_labels == 1)}")

    new_colors = colors.copy()
    dynamic_mask = pred_labels == 1
    edge_dynamic_mask = pred_labels == 2
    new_colors[dynamic_mask] = [1.0, 0.0, 0.0]
    new_colors[edge_dynamic_mask] = [0.0, 1.0, 0.0]

    visualized_pcd = o3d.geometry.PointCloud()
    visualized_pcd.points = o3d.utility.Vector3dVector(points)
    visualized_pcd.colors = o3d.utility.Vector3dVector(new_colors)

    if output_file_path is None:
        base_name = ply_file_path.split('.')[0]
        output_file_path = f"{base_name}_threshold_{threshold}.ply"

    o3d.io.write_point_cloud(output_file_path, visualized_pcd)
    print(f"Результат сохранен в: {output_file_path}")

In [ ]:
visualize_dynamic_points_with_threshold(
        "/content/combined_scene.ply",
        "/content/best_model.pth",
        threshold=0.4,
        voxel_size = 0.01
    )

Downsample включён, voxel_size=0.01
Загружено точек после препроцессинга: 2700040
Используемый порог: 0.3
Обработан батч 1/14
Обработан батч 2/14
Обработан батч 3/14
Обработан батч 4/14
Обработан батч 5/14
Обработан батч 6/14
Обработан батч 7/14
Обработан батч 8/14
Обработан батч 9/14
Обработан батч 10/14
Обработан батч 11/14
Обработан батч 12/14
Обработан батч 13/14
Обработан батч 14/14
Спорных точек определено: 486313
Статических точек: 2235785
Динамических точек: 110243
Спорных точек (на краю): 354012
Точек на полу: 1714180
Результат сохранен в: /content/combined_scene_threshold_0.3.ply


### MAIN

In [116]:
import torch
import numpy as np
import open3d as o3d
from models.pointnet2_sem_seg import get_model
import torch.nn.functional as F
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from collections import defaultdict

def remove_dynamic_points_with_threshold(ply_file_path, model_path, output_file_path=None,
                                          threshold=0.5, device=('cuda' if torch.cuda.is_available() else "cpu"), voxel_size=0.1,
                                          use_downsample=True, edge_distance_threshold=6.0,
                                          z_upper_static_threshold=6.5, ground_height_threshold=0.55, grid_divisions=15):
    model = get_model(num_classes=2).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    pcd = o3d.io.read_point_cloud(ply_file_path)

    if use_downsample:
        pcd = pcd.voxel_down_sample(voxel_size)
        print(f"Downsample включён, voxel_size={voxel_size}")
    else:
        print("Downsample отключён")

    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.2, max_nn=30))

    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors) if pcd.has_colors() else np.ones((len(points), 3)) * 0.5
    normals = np.asarray(pcd.normals)

    print(f"Загружено точек после препроцессинга: {len(points)}")
    print(f"Используемый порог: {threshold}")

    batch_size = 200000
    all_dynamic_probs = []

    total_batches = (len(points) + batch_size - 1) // batch_size

    for batch_idx in range(total_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, len(points))

        batch_points = points[start_idx:end_idx]
        batch_colors = colors[start_idx:end_idx]
        batch_normals = normals[start_idx:end_idx]

        xyz = batch_points.copy()
        xyz_mean = np.mean(xyz, axis=0, keepdims=True)
        xyz_centered = xyz - xyz_mean

        xyz_max = np.max(np.abs(xyz_centered))
        if xyz_max > 0:
            xyz_normalized = xyz_centered / (xyz_max + 1e-8)
        else:
            xyz_normalized = xyz_centered

        rgb = batch_colors.copy()
        if np.max(rgb) > 1.0:
            rgb = rgb / 255.0

        rgb_normalized = (rgb - np.mean(rgb, axis=0, keepdims=True)) / (np.std(rgb, axis=0, keepdims=True) + 1e-8)

        features = np.hstack([xyz_normalized, rgb_normalized, batch_normals])

        features_tensor = torch.tensor(features, dtype=torch.float32).unsqueeze(0)
        features_tensor = features_tensor.permute(0, 2, 1).to(device)

        with torch.no_grad():
            pred, _ = model(features_tensor)
            probas = F.softmax(pred, dim=2)
            dynamic_probs = probas[0, :, 1].cpu().numpy()
            all_dynamic_probs.append(dynamic_probs)

        print(f"Обработан батч {batch_idx + 1}/{total_batches}")

    all_dynamic_probs = np.concatenate(all_dynamic_probs)

    pred_labels = (all_dynamic_probs > threshold).astype(int)


    # статика для земли
    min_x, min_y = np.min(points[:, 0]), np.min(points[:, 1])
    max_x, max_y = np.max(points[:, 0]), np.max(points[:, 1])

    x_edges = np.linspace(min_x, max_x, grid_divisions + 1)
    y_edges = np.linspace(min_y, max_y, grid_divisions + 1)

    ground_mask = np.zeros(len(points), dtype=bool)

    for i in range(grid_divisions):
        for j in range(grid_divisions):
            x_min, x_max = x_edges[i], x_edges[i+1]
            y_min, y_max = y_edges[j], y_edges[j+1]

            cell_mask = (points[:, 0] >= x_min) & (points[:, 0] < x_max) & \
                       (points[:, 1] >= y_min) & (points[:, 1] < y_max)

            if np.sum(cell_mask) == 0:
                continue

            dynamic_in_cell = cell_mask & (pred_labels == 1)

            if np.sum(dynamic_in_cell) > 0:
                min_z_cell = np.min(points[dynamic_in_cell, 2])
            else:
                min_z_cell = np.min(points[cell_mask, 2])

            ground_in_cell = cell_mask & (points[:, 2] < (min_z_cell + ground_height_threshold))
            ground_mask[ground_in_cell] = True

    pred_labels[ground_mask] = 0


    # поиск граничных точек (грок)
    num_samples = min(1000, len(points))
    if num_samples > 0:
        np.random.seed(0)
        sample_idx = np.random.choice(len(points), num_samples, replace=False)
        sample_points_xy = points[sample_idx, :2]
        tree_sample = KDTree(sample_points_xy)
        dists, _ = tree_sample.query(sample_points_xy, k=2)
        avg_nn_dist = np.mean(dists[:, 1]) if num_samples > 1 else 0.0
    else:
        avg_nn_dist = 0.0

    grid_size = max(avg_nn_dist * 2.0, 1e-6)

    cell_points = defaultdict(list)
    for i in range(len(points)):
        p = points[i, :2]
        ix = int(p[0] / grid_size)
        iy = int(p[1] / grid_size)
        cell_points[(ix, iy)].append(i)

    occupied_cells = set(cell_points.keys())

    boundary_cells = set()
    for cell in occupied_cells:
        ix, iy = cell
        is_bound = False
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if dx == 0 and dy == 0:
                    continue
                neigh = (ix + dx, iy + dy)
                if neigh not in occupied_cells:
                    is_bound = True
                    break
            if is_bound:
                break
        if is_bound:
            boundary_cells.add(cell)

    is_boundary = np.zeros(len(points), dtype=bool)
    for cell in boundary_cells:
        for i in cell_points[cell]:
            is_boundary[i] = True

    # Для спорных точек
    non_ground_dynamic_idx = np.where((pred_labels == 1) & (~ground_mask))[0]
    if len(non_ground_dynamic_idx) > 0 and np.sum(is_boundary) > 0:
      boundary_idx = np.where(is_boundary)[0]
      boundary_tree = KDTree(points[boundary_idx, :2])
      dists_to_boundary, _ = boundary_tree.query(points[non_ground_dynamic_idx, :2])
      edge_mask = dists_to_boundary < edge_distance_threshold

      candidate_indices = non_ground_dynamic_idx[edge_mask]
      high_prob_mask = all_dynamic_probs[candidate_indices] < 0.6
      pred_labels[candidate_indices[high_prob_mask]] = 2

    if z_upper_static_threshold is not None:
        min_z = np.min(points[:, 2])
        upper_mask = points[:, 2] > (min_z + z_upper_static_threshold)
        pred_labels[upper_mask] = 2

    print(f"Статических точек: {np.sum(pred_labels == 0)}")
    print(f"Динамических точек: {np.sum(pred_labels == 1)}")

    static_mask = (pred_labels == 0) | (pred_labels == 2)
    points_static = points[static_mask]
    colors_static = colors[static_mask]

    visualized_pcd = o3d.geometry.PointCloud()
    visualized_pcd.points = o3d.utility.Vector3dVector(points_static)
    visualized_pcd.colors = o3d.utility.Vector3dVector(colors_static)

    if output_file_path is None:
        base_name = ply_file_path.split('.')[0]
        output_file_path = f"{base_name}_threshold_{threshold}.ply"

    o3d.io.write_point_cloud(output_file_path, visualized_pcd)
    print(f"Результат сохранен в: {output_file_path}")

In [115]:
remove_dynamic_points_with_threshold(
        "/content/combined_scene_1.ply",
        "/content/best_model.pth",
        threshold=0.4,
        voxel_size = 0.01
    )

get_model(
  (sa1): PointNetSetAbstraction(
    (mlp_convs): ModuleList(
      (0): Conv2d(12, 32, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
      (2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (mlp_bns): ModuleList(
      (0-1): 2 x BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (sa2): PointNetSetAbstraction(
    (mlp_convs): ModuleList(
      (0): Conv2d(67, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
    )
    (mlp_bns): ModuleList(
      (0-1): 2 x BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (sa3): PointNetSetAbstraction(
  

KeyboardInterrupt: 